# High-level TF Example

In [6]:
# Parameters
EPOCHS = 10
N_CLASSES=10
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9
GPU = True

In [7]:
import numpy as np
import os
import sys
import tensorflow as tf
from os import path
from utils import cifar_for_library, yield_mb
from nb_logging import NotebookLogger, output_to, error_to

In [8]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [9]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

In [10]:
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = "1"

In [11]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)

('OS: ', 'linux2')
('Python: ', '2.7.12 (default, Nov 19 2016, 06:48:10) \n[GCC 5.4.0 20160609]')
('Numpy: ', '1.13.1')
('Tensorflow: ', '1.3.0')


In [12]:
data_path = path.join(os.getenv('AZ_BATCHAI_INPUT_DATASET'), 'cifar-10-batches-py')

In [13]:
def create_symbol(training):
    """ TF pooling requires a boolean flag for dropout, faster when using
    'channels_first' for data_format """
    conv1 = tf.layers.conv2d(X, filters=50, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    relu1 = tf.nn.relu(conv1)
    conv2 = tf.layers.conv2d(relu1, filters=50, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    relu2 = tf.nn.relu(conv2)
    pool1 = tf.layers.max_pooling2d(relu2, pool_size=(2, 2), strides=(2, 2), 
                                    padding='valid', data_format='channels_first')
    drop1 = tf.layers.dropout(pool1, 0.25, training=training)
    
    conv3 = tf.layers.conv2d(drop1, filters=100, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    relu3 = tf.nn.relu(conv3)
    conv4 = tf.layers.conv2d(relu3, filters=100, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    relu4 = tf.nn.relu(conv4)
    pool2 = tf.layers.max_pooling2d(relu4, pool_size=(2, 2), strides=(2, 2), 
                                    padding='valid', data_format='channels_first')
    drop2 = tf.layers.dropout(pool2, 0.25, training=training)  
    
    flatten = tf.reshape(drop2, shape=[-1, 100*8*8])
    fc1 = tf.layers.dense(flatten, 512, activation=tf.nn.relu)
    drop3 = tf.layers.dropout(fc1, 0.5, training=training)
    logits = tf.layers.dense(drop3, N_CLASSES, name='output')
    return logits

In [14]:
def init_model(m):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.MomentumOptimizer(learning_rate=LR, momentum=MOMENTUM)
    training_op = optimizer.minimize(loss)
    return training_op

In [15]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Done.
((50000, 3, 32, 32), (10000, 3, 32, 32), (50000,), (10000,))
(dtype('float32'), dtype('float32'), dtype('int32'), dtype('int32'))
CPU times: user 784 ms, sys: 764 ms, total: 1.55 s
Wall time: 1.54 s


In [16]:
%%time
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, 3, 32, 32])
y = tf.placeholder(tf.int32, shape=[None])
training = tf.placeholder(tf.bool)  # Indicator for dropout layer
# Initialise model
sym = create_symbol(training)

CPU times: user 236 ms, sys: 0 ns, total: 236 ms
Wall time: 234 ms


In [17]:
%%time
model = init_model(sym)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 860 ms, sys: 4.14 s, total: 5 s
Wall time: 5.84 s


In [18]:
%%time
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label, training: True})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label, training: True})
    print(j, "Train accuracy:", acc_train)

(0, 'Train accuracy:', 0.40625)
(1, 'Train accuracy:', 0.46875)
(2, 'Train accuracy:', 0.65625)
(3, 'Train accuracy:', 0.640625)
(4, 'Train accuracy:', 0.609375)
(5, 'Train accuracy:', 0.734375)
(6, 'Train accuracy:', 0.734375)
(7, 'Train accuracy:', 0.78125)
(8, 'Train accuracy:', 0.90625)
(9, 'Train accuracy:', 0.765625)
CPU times: user 1min 1s, sys: 42.4 s, total: 1min 43s
Wall time: 2min 1s


In [19]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data, training: False})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 3.66 s, sys: 524 ms, total: 4.18 s
Wall time: 3.96 s


In [21]:
print("Accuracy: ", float(sum(y_guess == y_truth))/len(y_guess))

('Accuracy: ', 0.7826522435897436)
